In [1]:
import pandas as pd
import numpy as np
import json
import re

## Load dataset 

In [2]:
business_df = pd.read_csv('business.csv')
checkin_df = pd.read_csv('checkin.csv')

In [3]:
business_df['state'].unique()

array(['CA', 'MO', 'AZ', 'PA', 'TN', 'FL', 'IN', 'LA', 'AB', 'NV', 'ID',
       'DE', 'IL', 'NJ', 'NC', 'CO', 'WA', 'HI', 'UT', 'TX', 'MT', 'MI',
       'SD', 'XMS', 'MA', 'VI', 'VT'], dtype=object)

## Select the category containing 'food' or 'restaurant'

In [4]:
business_df = business_df[business_df['categories'].str.contains('Restaurants')|
                          business_df['categories'].str.contains('Food')|
                          business_df['categories'].str.contains('Restaurant')|
                          business_df['categories'].str.contains('Foods')
                         ]

In [5]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"{'WiFi': ""'free'"", 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."
150328,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,11177 N Oracle Rd,Oro Valley,AZ,85737,32.409552,-110.943073,4.5,41,1,"{'BusinessParking': ""{'garage': False, 'street...","Specialty Food, Food, Coffee & Tea, Coffee Roa...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-14:0', '..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
150339,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,DE,19014,39.856185,-75.427725,4.5,14,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Comfort Food, Food, Food Trucks, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


## Check and handle null values

In [6]:
business_df['state'].unique()

array(['PA', 'TN', 'MO', 'FL', 'IN', 'AB', 'NV', 'IL', 'AZ', 'LA', 'NJ',
       'CA', 'DE', 'ID', 'NC', 'CO', 'HI', 'MT', 'XMS'], dtype=object)

In [7]:
print(business_df.isnull().sum())

business_id        0
name               0
address          738
city               0
state              0
postal_code       26
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
attributes       877
categories         0
hours           9336
dtype: int64


#### Since the data set is large enough, just drop the null values

In [8]:
business_df = business_df.dropna(subset=['hours'])
business_df = business_df.dropna(subset=['attributes'])
business_df = business_df.dropna(subset=['address'])

In [9]:
business_df.shape

(54441, 14)

## Separate business hours by day

In [10]:
def format_time(time_str):
    if '-' in time_str:
        start, end = time_str.split('-')
        start_hour, start_min = start.split(':')
        end_hour, end_min = end.split(':')
        return f"{int(start_hour):02d}:{int(start_min):02d}-{int(end_hour):02d}:{int(end_min):02d}"
    else:
        parts = time_str.split(':')
        if len(parts) == 2:
            hour, minute = parts
        elif len(parts) == 3:
            hour, minute, _ = parts
        else:
            raise ValueError(f"Invalid time string format: {time_str}")
        return f"{int(hour):02d}:{int(minute):02d}"
def extract_hours(hours_str):
    if pd.isnull(hours_str):
        return None
    hours = json.loads(hours_str.replace("'", '"'))
    formatted_hours = {day: format_time(t) for day, t in hours.items()}
    return formatted_hours

def calc_hours(time_range):
    start, end = time_range.split('-')
    start_hr, start_min = map(int, start.split(':'))
    end_hr, end_min = map(int, end.split(':'))
    start_total = start_hr + start_min / 60
    end_total = end_hr + end_min / 60
    if end_total < start_total:
        end_total += 24
    return end_total - start_total

business_df['hours'] = business_df['hours'].apply(extract_hours)
business_df['hours_sum'] = business_df['hours'].apply(lambda d: sum(calc_hours(t) for t in d.values()) if isinstance(d, dict) else 0)
business_df['hours_mean'] = business_df['hours_sum'] / business_df['hours'].apply(lambda d: len(d) if isinstance(d, dict) else 0)

In [11]:
business_df['hours']

3         {'Monday': '07:00-20:00', 'Tuesday': '07:00-20...
4         {'Wednesday': '14:00-22:00', 'Thursday': '16:0...
5         {'Monday': '00:00-00:00', 'Tuesday': '06:00-22...
9         {'Monday': '00:00-00:00', 'Tuesday': '06:00-21...
12        {'Monday': '06:00-22:00', 'Tuesday': '06:00-22...
                                ...                        
150327    {'Monday': '00:00-00:00', 'Tuesday': '00:00-17...
150328    {'Monday': '00:00-00:00', 'Tuesday': '06:00-14...
150336    {'Monday': '11:00-22:00', 'Tuesday': '11:00-22...
150339    {'Monday': '00:00-00:00', 'Tuesday': '00:00-00...
150340    {'Monday': '11:00-22:00', 'Tuesday': '11:00-22...
Name: hours, Length: 54441, dtype: object

In [12]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
default_hours = '00:00-00:00'
for day in days:
    business_df[f'{day}_hours'] = business_df['hours'].apply(lambda x: x.get(day) if x else None)
    business_df[f'{day}_hours'] = business_df[f'{day}_hours'].fillna(default_hours)

In [13]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,hours,hours_sum,hours_mean,Monday_hours,Tuesday_hours,Wednesday_hours,Thursday_hours,Friday_hours,Saturday_hours,Sunday_hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,"{'Monday': '07:00-20:00', 'Tuesday': '07:00-20...",94.0,13.428571,07:00-20:00,07:00-20:00,07:00-20:00,07:00-20:00,07:00-21:00,07:00-21:00,07:00-21:00
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,"{'Wednesday': '14:00-22:00', 'Thursday': '16:0...",40.0,8.000000,00:00-00:00,00:00-00:00,14:00-22:00,16:00-22:00,12:00-22:00,12:00-22:00,12:00-18:00
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,"{'Monday': '00:00-00:00', 'Tuesday': '06:00-22...",90.0,12.857143,00:00-00:00,06:00-22:00,06:00-22:00,06:00-22:00,09:00-00:00,09:00-22:00,08:00-22:00
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,"{'Monday': '00:00-00:00', 'Tuesday': '06:00-21...",76.0,10.857143,00:00-00:00,06:00-21:00,06:00-21:00,06:00-16:00,06:00-16:00,06:00-17:00,06:00-21:00
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,...,"{'Monday': '06:00-22:00', 'Tuesday': '06:00-22...",112.0,16.000000,06:00-22:00,06:00-22:00,06:00-22:00,06:00-22:00,06:00-22:00,06:00-22:00,06:00-22:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,...,"{'Monday': '00:00-00:00', 'Tuesday': '00:00-17...",17.0,2.428571,00:00-00:00,00:00-17:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00
150328,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,11177 N Oracle Rd,Oro Valley,AZ,85737,32.409552,-110.943073,4.5,41,...,"{'Monday': '00:00-00:00', 'Tuesday': '06:00-14...",50.0,7.142857,00:00-00:00,06:00-14:00,06:00-14:00,06:00-14:00,06:00-14:00,06:00-15:00,06:00-15:00
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,...,"{'Monday': '11:00-22:00', 'Tuesday': '11:00-22...",74.0,12.333333,11:00-22:00,11:00-22:00,00:00-00:00,11:00-22:00,11:00-02:00,11:00-02:00,11:00-22:00
150339,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,DE,19014,39.856185,-75.427725,4.5,14,...,"{'Monday': '00:00-00:00', 'Tuesday': '00:00-00...",0.0,0.000000,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00


## Separate 'attributes' and keep only three features

In [14]:
def extract_attributes(attributes_str):
    if pd.isnull(attributes_str):
        return {}
    
    attributes = {}
    pattern = r"'([^']+)':\s*'([^']*)'"
    matches = re.findall(pattern, attributes_str)
    
    for match in matches:
        key, value = match
        if value.lower() in ['true', 'false']:
            attributes[key] = (value.lower() == 'true')
        elif value.isdigit():
            attributes[key] = int(value)
        else:
            attributes[key] = value
    
    return attributes

In [15]:
business_df['attributes'] = business_df['attributes'].apply(extract_attributes)

attributes_cols = ['BusinessAcceptsCreditCards', 'RestaurantsTakeOut', 'RestaurantsDelivery']

for col in attributes_cols:
    business_df[col] = business_df['attributes'].apply(lambda x: x.get(col))

In [16]:
business_df = business_df.dropna(subset=['BusinessAcceptsCreditCards'])
business_df = business_df.dropna(subset=['RestaurantsTakeOut'])
business_df = business_df.dropna(subset=['RestaurantsDelivery'])

In [17]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Monday_hours,Tuesday_hours,Wednesday_hours,Thursday_hours,Friday_hours,Saturday_hours,Sunday_hours,BusinessAcceptsCreditCards,RestaurantsTakeOut,RestaurantsDelivery
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,07:00-20:00,07:00-20:00,07:00-20:00,07:00-20:00,07:00-21:00,07:00-21:00,07:00-21:00,False,True,False
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,00:00-00:00,06:00-22:00,06:00-22:00,06:00-22:00,09:00-00:00,09:00-22:00,08:00-22:00,True,True,True
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,00:00-00:00,06:00-21:00,06:00-21:00,06:00-16:00,06:00-16:00,06:00-17:00,06:00-21:00,True,True,True
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,...,06:00-22:00,06:00-22:00,06:00-22:00,06:00-22:00,06:00-22:00,06:00-22:00,06:00-22:00,True,True,True
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,...,10:00-18:00,10:00-20:00,10:00-20:00,10:00-20:00,10:00-20:00,10:00-20:00,00:00-00:00,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,...,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,True,True,True
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,...,00:00-00:00,00:00-17:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,00:00-00:00,True,False,False
150328,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,11177 N Oracle Rd,Oro Valley,AZ,85737,32.409552,-110.943073,4.5,41,...,00:00-00:00,06:00-14:00,06:00-14:00,06:00-14:00,06:00-14:00,06:00-15:00,06:00-15:00,True,True,False
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,...,11:00-22:00,11:00-22:00,00:00-00:00,11:00-22:00,11:00-02:00,11:00-02:00,11:00-22:00,True,True,True


## Divide 'category' into several major categories

In [18]:
category_mapping = {
    'fast food': 'Fast Food',
    'fastfood': 'Fast Food',
    'burgers': 'Fast Food',
    'pizza': 'Fast Food',
    'sandwiches': 'Fast Food',

    'coffee': 'Coffee & Tea',
    'tea': 'Coffee & Tea',
    'cafe': 'Coffee & Tea',
    'cafes': 'Coffee & Tea',

    'bar': 'Bars & Nightlife',
    'bars': 'Bars & Nightlife',
    'pub': 'Bars & Nightlife',
    'pubs': 'Bars & Nightlife',
    'nightlife': 'Bars & Nightlife',

    'dessert': 'Bakeries & Desserts',
    'desserts': 'Bakeries & Desserts',
    'bakery': 'Bakeries & Desserts',
    'bakeries': 'Bakeries & Desserts',
    'cake': 'Bakeries & Desserts',
    'cakes': 'Bakeries & Desserts',
    'ice cream': 'Bakeries & Desserts',
    'frozen yogurt': 'Bakeries & Desserts',
    'donuts': 'Bakeries & Desserts',
    'bagels': 'Bakeries & Desserts',
    
    'chinese': 'Ethnic Cuisine',
    'italian': 'Ethnic Cuisine',
    'mexican': 'Ethnic Cuisine',
    'indian': 'Ethnic Cuisine',
    'japanese': 'Ethnic Cuisine',
    'korean': 'Ethnic Cuisine',
    'thai': 'Ethnic Cuisine',
    'vietnamese': 'Ethnic Cuisine',
    'asian fusion': 'Ethnic Cuisine',
    
    'salad': 'Healthy Food',
    'salads': 'Healthy Food',
    'vegan': 'Healthy Food',
    'vegetarian': 'Healthy Food',
    'gluten-free': 'Healthy Food',
    
    'seafood': 'Other Cuisine',
    'steak': 'Other Cuisine',
    'steakhouse': 'Other Cuisine',
    'bbq': 'Other Cuisine',
    'barbeque': 'Other Cuisine',
    'buffet': 'Other Cuisine',
    'diner': 'Other Cuisine',
    'diners': 'Other Cuisine',
    'juice': 'Other Food & Drink',
    'smoothies': 'Other Food & Drink',
    
}

def clean_category(category):
    category = category.lower() 
    category = re.sub(r'[^a-z]', '', category)
    return category_mapping.get(category, 'Other')

business_df['categories'] = business_df['categories'].astype(str)
business_df['categories'] = business_df['categories'].apply(lambda x: [clean_category(cat) for cat in x.split(', ')])

main_category_order = ['Ethnic Cuisine', 'Fast Food', 'Coffee & Tea', 'Bars & Nightlife', 'Bakeries & Desserts', 
                       'Healthy Food', 'Other Cuisine', 'Other Food & Drink', 'Other']

def select_main_category(categories):
    for category in main_category_order:
        if category in categories:
            return category
    return 'Other'

business_df['main_category'] = business_df['categories'].apply(select_main_category)

In [19]:
business_df['main_category'].unique()

array(['Bakeries & Desserts', 'Fast Food', 'Other Cuisine',
       'Ethnic Cuisine', 'Coffee & Tea', 'Other', 'Bars & Nightlife',
       'Healthy Food'], dtype=object)

## Count the number of checkins for each business

In [20]:
checkin_df.dropna(subset=['date'])

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,"2013-03-23 16:22:47, 2013-04-07 02:03:12, 2013..."
131926,zznZqH9CiAznbkV6fXyHWA,2021-06-12 01:16:12
131927,zzu6_r3DxBJuXcjnOYVdTw,"2011-05-24 01:35:13, 2012-01-01 23:44:33, 2012..."
131928,zzw66H6hVjXQEt0Js3Mo4A,"2016-12-03 23:33:26, 2018-12-02 19:08:45"


In [21]:
checkin_df['date'] = checkin_df['date'].apply(lambda x: x.strip('[]').split(', '))

checkin_df = checkin_df.explode('date')

checkin_counts = checkin_df.groupby(['business_id', 'date']).size().reset_index(name='count')

checkin_counts = checkin_counts.groupby('business_id')['count'].sum().reset_index()

In [22]:
checkin_counts

,business_id,count
0,---kPU91CF4Lq2-WlRu9Lw,11
1,--0iUa4sNDFiZFrAdIWhZQ,10
2,--30_8IhuyMHbSOcNWd6DQ,2
3,--7PUidqRWpRSpXebiyxTg,10
4,--7jw19RH9JKXgFohspgQw,26
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,67
131926,zznZqH9CiAznbkV6fXyHWA,1
131927,zzu6_r3DxBJuXcjnOYVdTw,23
131928,zzw66H6hVjXQEt0Js3Mo4A,2


## Join the two tables and final cleaning

In [23]:
clean_data = pd.merge(business_df, checkin_counts, on='business_id', how='left')

In [24]:
clean_data.isnull().sum()

business_id                     0
name                            0
address                         0
city                            0
state                           0
postal_code                     1
latitude                        0
longitude                       0
stars                           0
review_count                    0
is_open                         0
attributes                      0
categories                      0
hours                           0
hours_sum                       0
hours_mean                      0
Monday_hours                    0
Tuesday_hours                   0
Wednesday_hours                 0
Thursday_hours                  0
Friday_hours                    0
Saturday_hours                  0
Sunday_hours                    0
BusinessAcceptsCreditCards      0
RestaurantsTakeOut              0
RestaurantsDelivery             0
main_category                   0
count                         221
dtype: int64

In [25]:
clean_data = clean_data.dropna(subset=['count','postal_code'])
clean_data = clean_data[clean_data['state'] != 'AB']

In [26]:
clean_data.isnull().sum()

business_id                   0
name                          0
address                       0
city                          0
state                         0
postal_code                   0
latitude                      0
longitude                     0
stars                         0
review_count                  0
is_open                       0
attributes                    0
categories                    0
hours                         0
hours_sum                     0
hours_mean                    0
Monday_hours                  0
Tuesday_hours                 0
Wednesday_hours               0
Thursday_hours                0
Friday_hours                  0
Saturday_hours                0
Sunday_hours                  0
BusinessAcceptsCreditCards    0
RestaurantsTakeOut            0
RestaurantsDelivery           0
main_category                 0
count                         0
dtype: int64

In [27]:
clean_data.to_csv('clean_data.csv')